# Модуль 4.3 — Гибридный поиск (BM25 + эмбеддинги)

**Цель:** повысить точность поиска, комбинируя ключевые слова и семантику.

**Что сделаем:**
- установим библиотеки
- подготовим документы
- настроим BM25 + векторный поиск
- объединим результаты через EnsembleRetriever

In [ ]:
%pip -q install -U \
  langchain \
  langchain-community \
  langchain-openai \
  chromadb \
  rank-bm25 \
  pydantic==2.12.3 \
  requests==2.32.4

## Если Colab ругается на зависимости

Иногда в Colab установлены библиотеки с жёсткими версиями (например, `google-adk` и `opentelemetry`).
Если после установки появляются конфликты — зафиксируйте совместимые версии ниже.

In [ ]:
%pip -q install -U \
  opentelemetry-api==1.37.0 \
  opentelemetry-sdk==1.37.0 \
  opentelemetry-proto==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0 \
  opentelemetry-exporter-otlp-proto-grpc==1.37.0

# На случай, если в окружении уже стоит более новая версия
def _force_pins():
    import sys, subprocess
    pkgs = [
        "opentelemetry-api==1.37.0",
        "opentelemetry-sdk==1.37.0",
        "opentelemetry-proto==1.37.0",
        "opentelemetry-exporter-otlp-proto-common==1.37.0",
        "opentelemetry-exporter-otlp-proto-grpc==1.37.0",
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U", "--force-reinstall"] + pkgs)

_force_pins()

## Подготовка папки `data`

Если папки нет, создадим её и положим пример файла, чтобы всё запускалось сразу.

In [ ]:
from pathlib import Path

data_dir = Path("./data")
data_dir.mkdir(parents=True, exist_ok=True)

sample_path = data_dir / "sample.txt"
if not sample_path.exists():
    sample_path.write_text(
        "LangChain помогает строить цепочки (chains), агенты и RAG‑системы.\n"
        "RAG улучшает ответы, добавляя знания из документов.\n"
        "Эмбеддинги позволяют искать похожие фрагменты текста.",
        encoding="utf-8",
    )

print("Файлы в data:", [p.name for p in data_dir.glob("*.txt")])

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Базовые ретриверы: BM25 и векторный

BM25 ищет по ключевым словам, а векторный поиск — по смыслу.
Комбинация часто даёт лучший результат.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

loader = DirectoryLoader(
    "./data",
    glob="**/*.txt",
    loader_cls=TextLoader,
    show_progress=True,
)

documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

chunks = splitter.split_documents(documents)

# BM25 ретривер
bm25 = BM25Retriever.from_documents(chunks)
bm25.k = 3

# Векторный ретривер
store = LocalFileStore("./cache/embeddings")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url=os.environ.get("OPENAI_BASE_URL"))
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embeddings, store, namespace="openai")

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=cached_embedder,
    persist_directory="./db/chroma",
)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

## EnsembleRetriever: объединяем результаты

Мы комбинируем два ретривера и задаём веса. Это даёт баланс между точным совпадением и семанти

In [ ]:
from langchain.retrievers import EnsembleRetriever

ensemble = EnsembleRetriever(
    retrievers=[bm25, vector_retriever],
    weights=[0.5, 0.5],
)

query = "Что такое LangChain?"
results = ensemble.get_relevant_documents(query)

for i, doc in enumerate(results, 1):
    print(f"\nРезультат {i}:")
    print(doc.page_content[:400])

## Практические рекомендации

- Начните с весов 0.5/0.5 и подбирайте под домен.
- Если много точных терминов — увеличивайте вес BM25.
- Если контекст важнее ключевых слов — увеличивайте вес векторного поиска.
- Логируйте результаты и оценивайте качество на тестовых запросах.